<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']
%tensorflow_version 1.x
!pip install -r zalo/requirements.txt
!pip install kashgari

DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/model.h5"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

In [0]:
from QASystem.preprocess import ZaloDatasetProcessor
import kashgari
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)

tokenizer = bert_embedding.tokenizer
processor = ZaloDatasetProcessor()
processor.load_from_path(DATASET_PATH, "train")
processor.load_from_path(DATASET_PATH, "train")

data = processor.train_data
val = processor.val_data

for d in data:
  train = tokenizer.tokenize(d.get('question')) + tokenizer.tokenize(data.get('text'))[1:]
  label = d.get('label', False)

for d in val:
  val_x = tokenizer.tokenize(d.get('question')) + tokenizer.tokenize(data.get('text'))[1:]
  val_y = d.get('label', False)


In [0]:
from tensorflow.python import keras
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()

tf_board_callback = keras.callbacks.TensorBoard(log_dir='./logs', update_freq=1000)

# Build-in callback for print precision, recall and f1 at every epoch step
eval_callback = EvalCallBack(kash_model=model,
                             valid_x=valid_x,
                             valid_y=valid_y,
                             step=5)

model.fit(train_x,
          train_y,
          val_x,
          val_y,
          batch_size=100,
          callbacks=[eval_callback, tf_board_callback])

model.save(SAVE_MODEL_PATH)


In [0]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(Xtest, ytest, verbose=1)
print("loss: %f\n accruracy: %f\nf1_score: %f\nprecision: %f\n, recall: %f\n:" 
      % (loss, accurany,f1_score,precision, recall))